# Extract tiles
## Functions

In [1]:
import cv2
import numpy as np
from PIL import Image

def create_tile_from_Visium(ori_image: np.array, save_path: str, center_x: int, center_y: int, cut_size, output_size=None) -> None:
    upper = center_x - cut_size[0] // 2
    left = center_y - cut_size[1] // 2
    
    ### 02-12 update
    upper = max(0, upper)
    left = max(0, left)
    ###
    
    tile = ori_image[upper:upper+cut_size[0], left:left+cut_size[1], ::-1] # cv2 is BGR while input is RGB
    if output_size is not None:
        tile = cv2.resize(tile, dsize=(output_size[0], output_size[1]), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(save_path, tile)
    
def create_tile_from_coords(ori_image: np.array, save_path: str, top: int, left: int, tile_size: int) -> None:
    tile = ori_image[top:top + tile_size, left:left + tile_size, ::-1]  # RGB to BGR
    cv2.imwrite(save_path, tile)

def load_image(img_path):
    fp = open(img_path, 'rb')
    pic = Image.open(fp)
    pic = np.array(pic)
    fp.close()
    return pic

def load_full_image(img_path):
    Image.MAX_IMAGE_PIXELS = None
    original_image = load_image(img_path)
    return original_image

## If you want to extract tiles from spots

In [ ]:
import scanpy as sc

path_ST_adata = "/cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/sim/pseudo_adata_real.h5ad"
adata = sc.read_h5ad(path_ST_adata)
adata_name = 'Xenium_FFPE_Human_Breast_Cancer_Rep1'

print("Spot diameter:", adata.uns["spatial"][adata_name]["scalefactors"]["spot_diameter_fullres"])

/cluster/CBIO/home/lgortana/anaconda3/envs/plugin-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Spot diameter: 204.39928782218652


In [ ]:
import pandas as pd
import os

tile_save_dir = '/cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/histocell/BRCA/tiles/Rep1_256'
image_path = '/cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/pyr_tif/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_image.tif'
height, width = 256, 256

full_image = load_full_image(image_path)
print(full_image.shape)
image = full_image.copy()

# valid_positions = tissue_positions[tissue_positions['in_tissue'] == 1]
# print(f'##### Tiles Saved in {tile_save_dir} #####')
os.makedirs(tile_save_dir, exist_ok=True)

coords = adata.obsm["spatial"].astype("int64")
spot_ids = adata.obs["spot_id"]
valid_positions = pd.DataFrame({
    "barcode": spot_ids.values,
    "in_tissue": 1,
    "pxl_row_in_fullres": coords[:, 1],
    "pxl_col_in_fullres": coords[:, 0],
})

for _, coordinates in valid_positions.iterrows():
    spot_id, pixel_x, pixel_y = coordinates['barcode'], int(coordinates['pxl_row_in_fullres']), int(coordinates['pxl_col_in_fullres'])
    create_tile_from_Visium(full_image, os.path.join(tile_save_dir, f"{spot_id}.jpg"), center_x=pixel_x, center_y=pixel_y, cut_size=[height, width])

(20511, 27587, 3)


## If you want to extract tiles from an entire slide

In [ ]:
import csv
import os

tile_save_dir = '/cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/histocell/BRCA/tiles/Rep1_256_hires'
image_path = '/cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/pyr_tif/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_image.tif'
csv_path = os.path.join(tile_save_dir, 'tile_centroids.csv')
tile_size = 256

os.makedirs(tile_save_dir, exist_ok=True)

full_image = load_full_image(image_path)
img_height, img_width, _ = full_image.shape
print(f"Image shape: {full_image.shape}")

tile_id = 0
tile_coords = []

for top in range(0, img_height - tile_size + 1, tile_size):
    for left in range(0, img_width - tile_size + 1, tile_size):
        tile_name = f"tile_{tile_id:06d}"
        tile_filename = f"{tile_name}.jpg"
        save_path = os.path.join(tile_save_dir, tile_filename)
        create_tile_from_coords(full_image, save_path, top, left, tile_size)
        
        center_x = left + tile_size // 2
        center_y = top + tile_size // 2
        tile_coords.append([tile_name, center_x, center_y])
        
        tile_id += 1

print(f"Saved {tile_id} tiles in {tile_save_dir}")

# Write tile center coordinates to CSV
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=' ')
    writer.writerow(['tile_id', 'center_x', 'center_y'])
    writer.writerows(tile_coords)

print(f"Saved tile centers to {csv_path}")

Image shape: (20511, 27587, 3)
Saved 8560 tiles in /cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/histocell/BRCA/tiles/Rep1_256_hires
Saved tile centers to /cluster/CBIO/data1/lgortana/Xenium_FFPE_Human_Breast_Cancer_Rep1/histocell/BRCA/tiles/Rep1_256_hires/tile_centroids.csv
